In [1]:
from keras.layers import LSTM, Dense, Masking
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 

Using TensorFlow backend.


In [2]:
all_chars = ['\n', ' ', '!', '$', '(', ')', ',', '.', '1', '2', '3', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '¿', 'É', 'Ñ', 'á', 'é', 'í', 'ñ', 'ó', 'ú']

In [3]:
chars_to_indices = dict((c, i) for i, c in enumerate(all_chars))
indices_to_chars = dict((i, c) for i, c in enumerate(all_chars))
print(chars_to_indices)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, '1': 8, '2': 9, '3': 10, '9': 11, ':': 12, ';': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'Y': 36, 'Z': 37, 'a': 38, 'b': 39, 'c': 40, 'd': 41, 'e': 42, 'f': 43, 'g': 44, 'h': 45, 'i': 46, 'j': 47, 'l': 48, 'm': 49, 'n': 50, 'o': 51, 'p': 52, 'q': 53, 'r': 54, 's': 55, 't': 56, 'u': 57, 'v': 58, 'x': 59, 'y': 60, 'z': 61, '¡': 62, '¿': 63, 'É': 64, 'Ñ': 65, 'á': 66, 'é': 67, 'í': 68, 'ñ': 69, 'ó': 70, 'ú': 71}


In [4]:
len(all_chars)

72

In [5]:
dropout_p = 0.5
model = Sequential()
model.add(LSTM(200, batch_input_shape=(1, 1, len(all_chars)),
               return_sequences=True, name='lstm_1',
               dropout=dropout_p, 
               recurrent_dropout=dropout_p, stateful=True))
model.add(LSTM(200, 
               dropout=dropout_p, 
               recurrent_dropout=dropout_p, name='lstm_2', 
               return_sequences=True, stateful=True))
model.add(Dense(len(all_chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 1, 200)               218400    
_________________________________________________________________
lstm_2 (LSTM)                (1, 1, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (1, 1, 72)                14472     
Total params: 553,672
Trainable params: 553,672
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.load_weights('many_2_many_3.hdf5')

In [11]:
import numpy as np
def one_hot_phrase(phrase):
    one_hot = np.zeros([len(phrase), 1, len(all_chars)])
    for i, ch in enumerate(phrase):
        one_hot[i, 0, chars_to_indices[ch]] = 1
    return one_hot

# Ir ingresando de a un caracter

In [12]:
char_data = 'NOVIO: '
model.reset_states()
for i, ch in enumerate(char_data):
    input_data = one_hot_phrase(ch)
    prediction = model.predict(input_data)
    char_index = np.argmax(prediction)
    char_predicted = indices_to_chars[char_index]
    prob = prediction[0, 0, char_index]
    print(ch, char_data[0:i+1], char_predicted, prob)

N N O 0.9788068
O NO V 0.99869734
V NOV I 0.99446684
I NOVI O 0.6734117
O NOVIO : 0.9983255
: NOVIO:   0.9697636
  NOVIO:  ¿ 0.20366669


# Realimentando resultado

In [15]:
char_data = 'L'
model.reset_states()
print(char_data, end='')
for i in range(3000):
    input_data = one_hot_phrase(char_data)
    prediction = model.predict(input_data)
    char_index = np.argmax(prediction)
    char_predicted = indices_to_chars[char_index]
    prob = prediction[0, 0, char_index]
    # print(ch, char_data[0:i+1], char_predicted, prob)
    print(char_predicted, end='')
    if char_predicted=='$':
        break
    char_data = char_predicted

LEONARDO: Es la cunta. 
MUJER: (La deja) Hay es calloro! (Se le venta.) ¿Qué mistas alí? (Se estando.)

PADRE: ¡Ay es el campre!

NOVIO: (Alerga) ¡Ay!

MOZO 1: ¿A la las?

Perlo te vamos a te pedre
de delgra el amorarame a la mañodad,
y el fuerte corra,
como si me carazlo me espera
de en el azmolo de la cabeza,
delos de carallos de las prescas del arroyas y las carados.
NOVIA:
¡Ay que la carana del ana,
como siento de la lovia.
MUJER: (Llorando)
Mi niño sol muerta
del agua de luega
la tuerta la monda a la criada.

NOVIA: (Al mirado) No lo saber a la ienta.
MADRE:
Cula diente la lova.
MUJER: (Llorando)
LEsada, llera, en la como una mana.
MUCHACHA 1:
No de carro
los mundos suegules deligar en moliendo.

NOVIO: ¿Qué más da?

MADRE: (Al rico) Diere a la cuerpa en la mando en un espero de los horas del como do los tros llega.

NOVIO: ¿Es que te unaces?

MADRE: ¡Vamos! ¡Que sigun ton la cabeza!

NOVIA: (Seria) Apora la muno sienta.

MADRE: Tu puerto de mucho mu mido espera de matar entra mal